In [ ]:
#лексико графический порядок!!
#Читска на верхнем регистре уже есть
#

In [6]:
%%time 
%matplotlib inline
%pylab inline
from nltk.collocations import *
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# класс нормализации текста 

class Normalization:
    __tokens = []                               #массив из токенизированного текста
    
    def __init__(self, text, regular='[^a-zA-Z0-9.& ]', registr = False):
        reg = re.compile(regular)
        middleToken = []
        if not registr :                        #очистка строки по регулярке + удаление всех лишних пробелов
            self.text = re.sub(r"\s+", u" ",reg.sub(' ', text.strip()).lower())
        else:
            self.text = re.sub(r"\s+", u" ",reg.sub(' ', text.strip()))
        self.__tokens = word_tokenize(self.text)
        for token in self.__tokens:             #Формирование токенов из (word.word)
            if token.find(".") != -1 and token != ".":
                token = token.replace(".", " . ")
                middleToken+=word_tokenize(token)
            else:
                middleToken.append(token)
        self.__tokens = middleToken
    
    def printText(self):
        print(self.text)
        
    def printTokens(self):
        print(self.__tokens[0:900])
        
    #WordNet лемматизация, результат формуриуется в __tokens
    def lemmatisation(self): 
        wnl = nltk.WordNetLemmatizer()
        self.__tokens = [wnl.lemmatize(token) for token in self.__tokens]
        self.text = nltk.Text(self.__tokens)
        return self.__tokens
    
    #Очистка слов, по умолчанию стоит классический корпус стоп-слов из NLTK
    def wordCleaner(self, stopWords = stopwords.words('english')):
        self.__tokens = [w for w in self.__tokens if not w.lower() in stopWords]
        self.text = nltk.Text(self.__tokens)
        return self.__tokens
        
    #Удаление лишних частей речи
    def posTagging(self, badForm = ["PRP", "CD", "DT", "WRB"]):
        pos_tagger = nltk.pos_tag(self.__tokens)
        self.__tokens = [wt[0] for wt in pos_tagger if wt[1] not in badForm]
        self.text = nltk.Text(self.__tokens)
        return self.__tokens
        
    #Разбиение токенов на биграммы, используется ф-ция PMI (мб добавить несколько вариатнов?)
    def bigramMeasure(self, app_filter = 1):
        bigram_measures = nltk.collocations.BigramAssocMeasures() 
        finder = BigramCollocationFinder.from_words(self.__tokens) 
        finder.apply_freq_filter(app_filter)   
        self.__tokens = [t[0] for t in finder.score_ngrams (bigram_measures.pmi)]
        return self.__tokens
    
    #Удаление "мусора", использовать очень осторожно!! Удаляет все слова, встреча которых меньше min_entry
    def abraCleaner(self, min_entry = 1):
        dicToken = {}
        self.__tokens = [token for token in self.__tokens if len(token) > 1 or token == "."]
        for token in self.__tokens:
            if token.lower() not in dicToken:
                dicToken[token.lower()] = 1
            else: 
                dicToken[token.lower()] +=1 
        self.__tokens = [token for token in self.__tokens if dicToken[token.lower()] > min_entry]
        return self.__tokens
        
        
    #Полная нормализация текста, используются настройки ф-ций по умолчанию
    def fullNormalization(self, lem = True, clean = True, tag = True,
                          abra = True, bigram = True):
        if lem:
            self.lemmatisation()
        if clean:
            self.wordCleaner()
        if tag:
            self.posTagging()
        if abra:
            self.abraCleaner()
        if bigram:
            self.bigramMeasure()
        return self.__tokens
        

Populating the interactive namespace from numpy and matplotlib
CPU times: user 3.75 ms, sys: 0 ns, total: 3.75 ms
Wall time: 3.58 ms


In [2]:
s=""
for line in open("AllText.txt", "r"):
    s+=line

In [14]:
x = Normalization(s ,registr = True)
x.lemmatisation()
x.wordCleaner()
x.posTagging()
x.abraCleaner()
x.fullNormalization(bigram=False)
x.printTokens()

['investment', 'Preferred', 'Stock', 'involves', 'high', 'degree', 'risk', '.', 'making', 'investment', 'Preferred', 'Stock', 'carefully', 'consider', 'following', 'risk', 'well', 'information', 'contained', 'prospectus', '.', 'risk', 'described', 'could', 'materially', 'harm', 'business', 'financial', 'condition', 'result', 'operation', 'prospect', '.', 'result', 'trading', 'price', 'Preferred', 'Stock', 'could', 'decline', 'may', 'lose', 'part', 'investment', '.', 'statement', 'prospectus', 'including', 'statement', 'following', 'risk', 'factor', 'constitute', 'forward', 'looking', 'statement', '.', 'See', 'section', 'prospectus', 'entitled', 'Special', 'Note', 'Regarding', 'Forward', 'Looking', 'Statements', '.', 'Risks', 'Relating', 'CompanyWe', 'may', 'future', 'opportunity', 'participate', 'take', 'program', '.', 'able', 'obtain', 'policy', 'last', 'annual', 'LA', 'Citizens', 'take', 'occurring', 'December', 'year', 'approximately', 'policy', 'force', 'approximately', 'total', 'p

[]

In [14]:
s1 ="FGH, etf, fgh, Exchange Traded Funds"
y = Normalization(s1)
y.fullNormalization()

[('exchange', 'traded'),
 ('traded', 'fund'),
 ('etf', 'fgh'),
 ('fgh', 'etf'),
 ('fgh', 'exchange')]